In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import pymongo

In [2]:
# Path to chromedriver - Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/seidy/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [16]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.mars_db

## Visit the NASA mars news site

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

browser.visit(url)

In [5]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
browser.quit()

In [8]:
title_results = soup.find_all("li", class_=["slide"])

In [10]:
title = title_results[0].find("div", class_=["content_title"]).text

In [12]:
# .find() the paragraph text
paragraph_text = soup.find(class_="article_teaser_body").text

In [17]:
mars_result = {
    "title": title,
    "text": paragraph_text
}

db.mars_db.insert_one(mars_result)

## JPL Space Images Featured Image

In [18]:
# Path to chromedriver - Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/seidy/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [19]:
# Visit JPL space images Mars URL 
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"

In [20]:
browser.visit(jpl_url)

In [21]:
# Find and click the full image button - must be in all caps
featured_img_click = browser.links.find_by_partial_text('FULL').click()

In [22]:
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

In [24]:
# find the image url
header_img = jpl_soup.find_all('div', class_='floating_text_area')[0].find('a')['href']

header_img

'image/featured/mars3.jpg'

In [25]:
base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'

In [26]:
# Use the base url to create an absolute url
img_url = f'{base_url}{header_img}'

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

In [27]:
img_title = "Featured Image"
img_text = "Featured Image"

In [28]:
mars_result = {
    "title": img_title,
    "text": img_text,
    "link": img_url
}

db.mars_db.insert_one(mars_result)

In [31]:
browser.quit()

## Mars Facts

In [32]:
# Create a dataframe from the space-facts.com mars page
mars_facts_url = 'https://space-facts.com/mars/'

In [33]:
mars_table = pd.read_html(mars_facts_url)
mars_df = mars_table[0]

In [34]:
# clean the dataframe and export to HTML
mars_df_columns = mars_df.rename(columns={0:"Attribute", 1:"Information"})

In [35]:
mars_df_clean = mars_df_columns.set_index(keys="Attribute")

In [36]:
mars_df_clean = mars_df_clean.reset_index()

In [37]:
mars_df_clean

,Attribute,Information
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [40]:
len(mars_df_clean)

9

In [42]:
mars_df_clean["Attribute"][0]

'Equatorial Diameter:'

In [44]:
for row in range(0, len(mars_df_clean)):
    mars_result = {
        "attribute": mars_df_clean["Attribute"][row],
        "information": mars_df_clean["Information"][row]
    }

    db.mars_db.insert_one(mars_result)

## Hemispheres

In [45]:
# Path to chromedriver - Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/seidy/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [46]:
# visit the USGS astrogeology page for hemisphere data from Mars
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [47]:
browser.visit(hemi_url)

In [48]:
hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [50]:
# finding length of all divs in hemisphere for list
num_of_hemi = len(hemi_soup.find_all('div', class_='item'))

In [51]:
# lists to append to:
hemi_hrefs = []

# for loop to extract information

for i in range(0,num_of_hemi):
    
    # navigating browser remotely and creating partial_text to search for with click
    partial_text = hemi_soup.find_all('div', class_='item')[i].text[0:15]
    browser.links.find_by_partial_text(partial_text).click()
    inner_html = browser.html
    inner_soup = BeautifulSoup(inner_html, 'html.parser')
    
    # finding hrefs and appending to list
    inner_href = inner_soup.find('div', class_='downloads').find('li').find('a')['href']
    
    # finding titles and appending to list
    inner_title = inner_soup.find('h2', class_='title').text
    
    mars_result = {
        'hemisphere': inner_title,
        'hemi_url': inner_href
    }
    
    hemi_hrefs.append(mars_result)
    
    db.mars_db.insert_one(mars_result)
    
    # navigating back to browser
    browser.back()

In [52]:
hemi_hrefs

[{'hemisphere': 'Cerberus Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  '_id': ObjectId('606a65d3404e57159c250a39')},
 {'hemisphere': 'Schiaparelli Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  '_id': ObjectId('606a65d6404e57159c250a3a')},
 {'hemisphere': 'Syrtis Major Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  '_id': ObjectId('606a65d8404e57159c250a3b')},
 {'hemisphere': 'Valles Marineris Hemisphere Enhanced',
  'hemi_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  '_id': ObjectId('606a65d9404e57159c250a3c')}]

In [53]:
browser.quit()